In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import syllables
import re

In [3]:
df = pd.read_csv('../Data/train.csv')

In [4]:
excerpts = df['excerpt']

In [5]:
i = 0
excerptData = {}
for entry in excerpts:
    count = 0
    excerptData[i] = [len(entry.split()), len(re.split(r'\.!?', entry))]
    for word in entry.split():
        word = word.strip(',.')
        if (syllables.estimate(word) >= 3) and ((not word.endswith('ed')) or (not word.endswith('ing')) or (not word.endswith('es'))):
            count += 1
    excerptData[i].append(count)
    i += 1
        

In [7]:
def GunningFog(words, sentences, complexWords):
    
    return (.4*((words/sentences) + 100*(complexWords/words)))

In [8]:
for i in range(len(excerptData)):
    excerptData[i].append(GunningFog(*excerptData[i]))

In [9]:
excerptData = np.array(list(excerptData.values()))